<a href="https://colab.research.google.com/github/akshu-2004/Tree-specie-classification/blob/main/efficient_tree_model_h5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# Step 1: Import Libraries
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Step 2: Define paths and image parameters
data_dir = '/content/drive/MyDrive/Tree_Species_Dataset'
img_height, img_width = 224, 224
batch_size = 32

# Step 3: ImageDataGenerators with auto validation split
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Step 4: Load MobileNetV2 without top layer
base_model = MobileNetV2(input_shape=(img_height, img_width, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

# Step 5: Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Step 6: Compile model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 7: Train model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)

# Step 8: Save model
model.save('mobilenetv2_tree_species.h5')
print("Model saved as mobilenetv2_tree_species.h5")


Found 612 images belonging to 31 classes.
Found 151 images belonging to 31 classes.
Epoch 1/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 159s 8s/step - accuracy: 0.0422 - loss: 4.0979 - val_accuracy: 0.1325 - val_loss: 3.2046
Epoch 2/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 31s 2s/step - accuracy: 0.1043 - loss: 3.6117 - val_accuracy: 0.1854 - val_loss: 2.8876
Epoch 3/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.1019 - loss: 3.3324 - val_accuracy: 0.1987 - val_loss: 2.6819
Epoch 4/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.1609 - loss: 3.0466 - val_accuracy: 0.2119 - val_loss: 2.5355
Epoch 5/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 30s 2s/step - accuracy: 0.1710 - loss: 2.9054 - val_accuracy: 0.2185 - val_loss: 2.4159
Epoch 6/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - accuracy: 0.2364 - loss: 2.7688 - val_accuracy: 0.2649 - val_loss: 2.3138
Epoch 7/20
20/20 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.2434 - loss: 2.5775 - val_accuracy: 0.3046 - val_loss: 2.2248
Epoch 8/20
20/20 ━━━━━━━━━━━━━━━━

Model saved as mobilenetv2_tree_species.h5
